# Практическое задание 1

## Данные о студенте

1. **ФИО**: Мироненко Александр Витальевич
2. **Факультет**: Мехмат
3. **Курс**: 4
4. **Группа**: 408

## Замечания

* Название ноутбука с реализацией должно иметь шаблон "**Prac01_Ivanov.ipynb**" и посылаться на почту mlcoursemm@gmail.com с темой **[CV2021:Prac01]**
* Дедлайн будет оговорен в чате курса
* Соблюдаем кодекс чести (по нулям и списавшему, и давшему списать)
* Можно (и нужно!) применять для реализации только библиотеку **Numpy**
* Ничего, крому Numpy, нельзя использовать для реализации 
* **Keras** используется только для тестирования Вашей реализации
* Если какой-то из классов не проходит приведенные тесты, то соответствующее задание не оценивается
* Возможно использование дополнительных (приватных) тестов
 

In [282]:
# Вам понадобится для реализации
import numpy as np
# Нужно для тестирования
from tensorflow import keras
import keras.layers as layers

* Вспомогательные функции для тестирования

In [283]:
def compare_tensors(x, y, tol=0.001, test_name='Test'):
  assert (x.shape == y.shape), test_name + ' different shapes'
  diff = np.sum((y - x)**2)
  assert (diff < tol), test_name + ' Failed!'
  print (test_name + ' Passed!')
  return

In [284]:
def compare_tensors_array(x, y, tol=0.001, test_name='Test'):
  assert (len(x) == len(y)), test_name + ' different lengths'
  for i in range(len(x)):
    t = test_name + ' subtest ' + str(i)
    compare_tensors(x[i], y[i], tol=tol, test_name=t)
  print (test_name + ' Passed!')
  return

* Шаблон класса любой операции (слоя), которую Вам необходимо будет реализовать

In [285]:
class Layer(object):
    def __init__(self):
        self.name = 'Layer'       
    def forward(self, input_data):
        pass

* (1 балл) Реализация "спрямляющего" слоя Flatten

In [286]:
class FlattenLayer(Layer):
    def __init__(self):
        self.name = 'Flatten'
    def forward(self, input_data):
      # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
      # Преобразуем в двухмерный тензор: при этом по первой размерности НЕ преобразуем
      # Выкладываем данные: сначала по последней размерности, затем по предпоследней и т.д.
      # Нужно заполнить Numpy-тензор out
        out = np.empty((input_data.shape[0],input_data[0].size))
        
        out[0] = input_data[0].flatten().reshape(1,input_data[0].size)
        
        for i in range(1,input_data.shape[0]):
            out[i] = input_data[i].flatten().reshape(1,input_data[0].size)  
        return out

* Функция предварительного тестирования слоя **Flatten**
* Функции с названием "**test_**" не менять
* Вы можете самостоятельно поиграться с параметрами типа B/C/H/W etc

In [287]:
def test_FlattenLayer():
  B = 1
  C = 1
  H = 3
  W = 3
  x = np.random.randn(B, C, H, W)
  y = layers.Flatten(data_format='channels_last')
  y_keras = y(x).numpy()
  y_out = FlattenLayer().forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Flatten 1')
  B = 2
  C = 2
  H = 3
  W = 3
  x = np.random.randn(B, C, H, W)

  y = layers.Flatten(data_format='channels_last')
  y_keras = y(x).numpy()

  y_out = FlattenLayer().forward(x)

  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Flatten 2')
  return

* Запуск теста слоя Flatten
* Нужно, чтобы все тесты были '*Passed!*'

In [288]:
test_FlattenLayer()

Test Flatten 1 Passed!
Test Flatten 2 Passed!


Я не особо понял, как исправлять на GPU.Но я попробовал запустить локально и на google collab, и везде выдавался правильный ответ, только если выбран  'channels_last'.

Я проверял на массивах вручную, вроде работает верно=)



---



* (1 балл) Реализация слоя субдискретизации **Global Average Pooling**

In [289]:
class GAP2DLayer(Layer):
    def __init__(self):
      self.name = 'GAP2D'
    def forward(self, input_data):
      # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
      # Сворачиваем по двум последним размерностям (то есть на выходе - минус две размерности)
      # Нужно заполнить Numpy-тензор out 
        out = np.empty([])
        out = input_data.mean(axis = (2,3))
        return out

In [290]:
def test_GAP2DLayer():
  B = 1
  C = 1
  H = 3
  W = 3
  x = np.random.randn(B, C, H, W)
  y = layers.GlobalAveragePooling2D(data_format='channels_first')
  y_keras = y(x).numpy()
  y_out = GAP2DLayer().forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test GAP2D 1')
  B = 1
  C = 2
  H = 3
  W = 3
  x = np.random.randn(B, C, H, W)
  y = layers.GlobalAveragePooling2D(data_format='channels_first')
  y_keras = y(x).numpy()
  y_out = GAP2DLayer().forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test GAP2D 2')
  return

In [291]:
test_GAP2DLayer()

Test GAP2D 1 Passed!
Test GAP2D 2 Passed!




---



* (2 балла) Реализация слоя субдискретизации **MaxPooling**

In [292]:
class MaxPool2DLayer(Layer):
    def __init__(self, pool_size=2, stride=2):
      self.name = 'MaxPool2D'
      self.pool_size = pool_size
      self.stride = stride
    def forward(self, input_data):
      # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
      # Нужно заполнить Numpy-тензор out 
        out = np.empty((input_data.shape[0], input_data.shape[1], 1+(input_data.shape[2]-self.pool_size)// self.stride,1+ (input_data.shape[3]-self.pool_size)// self.stride))
        
        for i in range(input_data.shape[0]):
            for j in range(input_data.shape[1]):
                for k in range(0,input_data.shape[2],self.stride):
                    for e in range(0,input_data.shape[3],self.stride):
                        
                        if k+self.pool_size >input_data.shape[2] or e+self.pool_size> input_data.shape[3]:
                            continue
                        out[i,j,k // self.stride,e // self.stride] = input_data[i, j, k:(k+self.pool_size), e:(e+self.pool_size)].max()
        return out

In [293]:
def test_MaxPool2DLayer():
  B = 1
  C = 1
  H = 4
  W = 4
  pool_size = 2
  stride = 2
  x = np.random.randn(B, C, H, W)
  y = layers.MaxPooling2D(pool_size=pool_size, strides=stride, padding="valid", data_format='channels_first')
  y_keras = y(x).numpy()
  y_out = MaxPool2DLayer(pool_size=pool_size, stride=stride).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test MaxPool2D 1')
  B = 2
  C = 2
  H = 3
  W = 3
  pool_size = 2
  stride = 1  
  x = np.random.randn(B, C, H, W)
  y = layers.MaxPooling2D(pool_size=pool_size, strides=stride, padding="valid", data_format='channels_first')
  y_keras = y(x).numpy()
  y_out = MaxPool2DLayer(pool_size=pool_size, stride=stride).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test MaxPool2D 2')
  return

In [294]:
test_MaxPool2DLayer()

Test MaxPool2D 1 Passed!
Test MaxPool2D 2 Passed!




---



* (3 балла) Реализация слоя **активации** (поддерживаются **relu**, **sigmoid**, **softmax**)

In [295]:
class ActivationLayer(Layer):
    def __init__(self, activation='relu'):
      # Активация (поддерживаем 'relu', 'sigmoid', 'softmax')
      self.name = 'Activation'
      self.activation = activation
    def forward(self, input_data):   
      # На входе:
      # четырехмерный тензор вида [batch, input_channels, height, width] для 'relu', 'sigmoid'
      # или двухмерный тензор вида [batch, logits]
      # SoftMax применяется по последней размерности
      # Нужно заполнить Numpy-тензор out 
      out = np.empty(input_data.shape)
      if self.activation == 'relu':
        relu = lambda x: np.maximum(0,x)
        out = np.array(list(map(relu,input_data)))
      if self.activation == 'sigmoid':
        out = 1/(1+np.exp(-input_data))
      if self.activation == 'softmax':
        out = np.exp(input_data)
        out = out/np.sum(out,axis = 1).reshape(input_data.shape[0],1)

      return out

In [296]:
def test_ActivationLayer():
  B = 1
  C = 1
  H = 4
  W = 4
  activation = 'relu'
  x = np.random.randn(B, C, H, W)
  y = layers.Activation(activation)
  y_keras = y(x).numpy()
  y_out = ActivationLayer(activation=activation).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Activation 1')
  B = 2
  C = 2
  H = 3
  W = 3
  activation = 'sigmoid'  
  x = np.random.randn(B, C, H, W)
  y = layers.Activation(activation)
  y_keras = y(x).numpy()
  y_out = ActivationLayer(activation=activation).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Activation 2')
  B = 3
  C = 10
  activation = 'softmax'
  x = np.random.randn(B, C)
  y = layers.Activation(activation)
  y_keras = y(x).numpy()
  y_out = ActivationLayer(activation=activation).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Activation 3')  
  return

In [297]:
test_ActivationLayer()

Test Activation 1 Passed!
Test Activation 2 Passed!
Test Activation 3 Passed!




---



* (3 балла) Реализация слоя пакетной нормализации **BatchNorm** (как для режима train, так и для режима test)

In [298]:
# Hint
# Train mode:
# out = (batch - mean(batch)) / sqrt(var(batch) + epsilon) * gamma + beta
# moving_mean = moving_mean * momentum + mean(batch) * (1 - momentum)
# moving_var = moving_var * momentum + var(batch) * (1 - momentum)
# Test mode:
# (batch - moving_mean) / sqrt(moving_var + epsilon) * gamma + beta

class BatchNormLayer(Layer):
    def __init__(self, momentum=0.99, epsilon=0.001, beta_init=None, gamma_init=None,
                 moving_mean_init=None, moving_var_init=None,
                 mode='train', input_channels=2):
      # mode: 'train', 'test'
      # Параметры gamma, beta, mean, var - все имеют размерность по количеству карт input_channels   
      self.name = 'BatchNorm'
      self.momentum = momentum
      self.epsilon = epsilon
      self.beta = beta_init
      self.gamma = gamma_init
      self.moving_mean = moving_mean_init
      self.moving_var = moving_var_init
      self.mode = mode
      self.input_channels = input_channels
    def forward(self, input_data):   
      # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
      # 1) Нужно заполнить Numpy-тензор out (той же размерности, что и вход)
      # 2) Нужно обновить moving_mean и moving_var в режиме 'train'
      out = np.empty(input_data.shape)
      if self.mode == 'train':
        for i in range(input_data.shape[1]):
          mean = input_data[:,i,:,:].mean()
          var = input_data[:,i,:,:].var()

          self.moving_mean[i] = self.moving_mean[i] * self.momentum + mean * (1 - self.momentum)

          self.moving_var[i] = self.moving_var[i] * self.momentum + var * (1 - self.momentum)

          
          out[:,i,:,:] = (input_data[:,i,:,:] - mean)/np.sqrt(var + self.epsilon)*self.gamma[i]+self.beta[i]
      
      if self.mode == 'test':
        for i in range(input_data.shape[1]):
          out[:,i,:,:] = (input_data[:,i,:,:] - self.moving_mean[i])/np.sqrt(self.moving_var[i] + self.epsilon) * self.gamma[i] + self.beta[i] 


      return out

In [299]:
def test_BatchNormLayer():
  B = 2
  C = 2
  H = 4
  W = 4
  beta_init = 0 * np.ones(C)
  gamma_init = 1 * np.ones(C)
  moving_mean_init = 0 * np.ones(C)
  moving_var_init= 1 * np.ones(C)
  momentum = 0.99
  epsilon = 0.001
  mode = 'train'
  x = np.random.randn(B, C, H, W)
  y = layers.BatchNormalization(axis=1, momentum=momentum, epsilon=epsilon, trainable=True)
  y_keras = y(x, training=True).numpy()
  y.set_weights([gamma_init, beta_init, moving_mean_init, moving_var_init])
  y_keras = y(x, training=True).numpy()
  y_out_layer = BatchNormLayer(momentum=momentum, epsilon=epsilon, beta_init=beta_init, gamma_init=gamma_init,
                 moving_mean_init=moving_mean_init, moving_var_init=moving_var_init,
                 mode=mode, input_channels=C)
  y_out = y_out_layer.forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test BatchNorm 1')
  compare_tensors_array(y.get_weights(), 
                        [y_out_layer.gamma, y_out_layer.beta, y_out_layer.moving_mean, y_out_layer.moving_var],
                        tol=0.00001, test_name='Test BatchNorm 1.1')
                        
  B = 2 
  C = 2 
  H = 4 
  W = 4 
  beta_init = 1 * np.ones(C)
  gamma_init = 0 * np.ones(C)
  moving_mean = 0 * np.ones(C)
  moving_var = 1 * np.ones(C)

  moving_mean_init = 0 * np.ones(C)## ДОБАВИЛ САМ!!!
  moving_var_init= 1 * np.ones(C)## ДОБАВИЛ САМ!!!

  momentum = 0.99
  epsilon = 0.001
  mode = 'test'
  x = np.random.randn(B, C, H, W)
  y = layers.BatchNormalization(axis=1, momentum=momentum, epsilon=epsilon, trainable=False)
  y_keras = y(x, training=False).numpy()
  y.set_weights([gamma_init, beta_init, moving_mean, moving_var])
  y_keras = y(x, training=False).numpy()
  y_out_layer = BatchNormLayer(momentum=momentum, epsilon=epsilon, beta_init=beta_init, gamma_init=gamma_init,
                 moving_mean_init=moving_mean_init, moving_var_init=moving_var_init,
                 mode=mode, input_channels=C)
  y_out = y_out_layer.forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test BatchNorm 2')  
  compare_tensors_array(y.get_weights(), 
                        [y_out_layer.gamma, y_out_layer.beta, y_out_layer.moving_mean, y_out_layer.moving_var],
                        tol=0.00001, test_name='Test BatchNorm 2.1')
  return

**На GOOGLE COLLAB этот тест не проходил без обнуления init коэффицентов по новому, не считаю это своей ошибкой**

Новый экземпляр класса получал старые коэффиценты mean и var


In [300]:
test_BatchNormLayer()

Test BatchNorm 1 Passed!
Test BatchNorm 1.1 subtest 0 Passed!
Test BatchNorm 1.1 subtest 1 Passed!
Test BatchNorm 1.1 subtest 2 Passed!
Test BatchNorm 1.1 subtest 3 Passed!
Test BatchNorm 1.1 Passed!
Test BatchNorm 2 Passed!
Test BatchNorm 2.1 subtest 0 Passed!
Test BatchNorm 2.1 subtest 1 Passed!
Test BatchNorm 2.1 subtest 2 Passed!
Test BatchNorm 2.1 subtest 3 Passed!
Test BatchNorm 2.1 Passed!




---



* (1 балл) Реализация **полносвязного** слоя

In [301]:
class DenseLayer(Layer):
    def __init__(self, input_dim, output_dim, W_init=None, b_init=None):
      self.name = 'Dense'
      self.input_dim = input_dim
      self.output_dim = output_dim
      self.W = W_init
      self.b = b_init
    def forward(self, input_data):
      # На входе - двухмерный тензор вида [batch, input_channels]
      # Работаем по второй размерности, по первой размерности НЕ преобразуем
      # Вначале нужно проверить на согласование размерностей входных данных и ядра!
      # Нужно заполнить Numpy-тензор out 
      out = np.empty((input_data.shape[0],self.output_dim))

      assert input_data.shape[1] == self.input_dim, "alert!!! different shapes"

      out = input_data.dot(self.W) + self.b
      return out

In [302]:
def test_DenseLayer():
  B = 1
  C_IN = 10
  C_OUT = 5
  x = np.random.randn(B, C_IN)
  W_init = np.random.randn(C_IN, C_OUT)
  b_init = np.random.randn(C_OUT)
  y = layers.Dense(C_OUT, use_bias=True)
  y_keras = y(x).numpy()
  y.set_weights([W_init, b_init])
  y_keras = y(x).numpy()
  y_out = DenseLayer(C_IN, C_OUT, W_init=W_init, b_init=b_init).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Dense 1')
  B = 2
  C_IN = 5
  C_OUT = 10
  x = np.random.randn(B, C_IN)
  W_init = np.random.randn(C_IN, C_OUT)
  b_init = np.random.randn(C_OUT)
  y = layers.Dense(C_OUT, use_bias=True, input_shape=(C_IN,))
  y_keras = y(x).numpy()
  y.set_weights([W_init, b_init])
  y_keras = y(x).numpy()
  y_out = DenseLayer(C_IN, C_OUT, W_init=W_init, b_init=b_init).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Dense 2')
  return

In [303]:
test_DenseLayer()

Test Dense 1 Passed!
Test Dense 2 Passed!




---



* (2 балла) Реализация **сверточного** слоя

In [304]:
class Conv2DLayer(Layer):
    def __init__(self, kernel_size=3, input_channels=2, output_channels=3, 
                 padding='same', stride=1, K_init=None, b_init=None):
      # padding: 'same' или 'valid'
      # Работаем с квадратными ядрами, поэтому kernel_size - одно число
      # Работаем с единообразным сдвигом, поэтому stride - одно число
      # Фильтр размерности [kernel_size, kernel_size, input_channels, output_channels]
      self.name = 'Conv2D'
      self.kernel_size = kernel_size
      self.input_channels = input_channels
      self.output_channels = output_channels
      self.kernel = K_init
      self.bias = b_init
      self.padding = padding
      self.stride = stride
    def forward(self, input_data):
      # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
      # Вначале нужно проверить на согласование размерностей входных данных и ядра!
      # Нужно заполнить Numpy-тензор out
      assert input_data.shape[1] == self.input_channels, "alert!!! different shapes"
      out = np.empty([])
      if self.padding == "valid":

        out = np.zeros((input_data.shape[0],self.output_channels,1 + (input_data.shape[2]-self.kernel_size)// self.stride,1 + (input_data.shape[3]-self.kernel_size)//self.stride))
        for out_k in range(self.output_channels):
          for batch in range(input_data.shape[0]):
            for i in range(0,input_data.shape[2] - self.kernel_size+1,self.stride):
              for j in range(0,input_data.shape[3]- self.kernel_size+1,self.stride):
                for in_k in range(self.input_channels):
          
                  out[batch,out_k,i // self.stride,j // self.stride] += np.sum(self.kernel[:,:,in_k,out_k] * input_data[batch,in_k,i : (i + self.kernel_size),j:(j + self.kernel_size)])

            out[batch,out_k] += self.bias[out_k]
      
      if self.padding == "same":
        input_data_with_pad = np.zeros((input_data.shape[0],self.input_channels,input_data.shape[2]+self.kernel_size -1,input_data.shape[3]+self.kernel_size-1))
        ##расширяем
        padingg = (self.kernel_size -1)//2
        for batch in range(input_data.shape[0]):
          for in_k in range(self.input_channels):
            input_data_with_pad[batch,in_k,padingg:-padingg,padingg:-padingg] = input_data[batch,in_k]
  
        input_data = input_data_with_pad
        out = np.zeros((input_data.shape[0],self.output_channels,1 + (input_data.shape[2]-self.kernel_size)// self.stride,1 + (input_data.shape[3]-self.kernel_size)//self.stride))
        for out_k in range(self.output_channels):
          for batch in range(input_data.shape[0]):
            for i in range(0,input_data.shape[2] - self.kernel_size+1,self.stride):
              for j in range(0,input_data.shape[3]- self.kernel_size+1,self.stride):
                for in_k in range(self.input_channels):
          
                  out[batch,out_k,i // self.stride,j // self.stride] += np.sum(self.kernel[:,:,in_k,out_k] * input_data[batch,in_k,i : (i + self.kernel_size),j:(j + self.kernel_size)])

            out[batch,out_k] += self.bias[out_k]

      return out

In [305]:
def test_Conv2DLayer():
  B = 1
  C_IN = 1
  C_OUT = 1
  H = 10
  W = 10
  K = 3
  S = 1
  padding = 'same'
  x = np.random.randn(B, C_IN, H, W)
  K_init = np.random.randn(K, K, C_IN, C_OUT)
  b_init = np.random.randn(C_OUT)
  y = layers.Conv2D(C_OUT, K, strides=S, padding=padding, data_format='channels_first',
    dilation_rate=1, groups=1, activation=None, use_bias=True)
  y_keras = y(x).numpy()
  y.set_weights([K_init, b_init])
  y_keras = y(x).numpy()
  y_out = Conv2DLayer(kernel_size=K, input_channels=C_IN, output_channels=C_OUT, 
                 padding=padding, stride=S, K_init=K_init, b_init=b_init).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Conv2D 1')
  B = 2
  C_IN = 3
  C_OUT = 5
  H = 9
  W = 9
  K = 3
  S = 2
  padding = 'valid'
  x = np.random.randn(B, C_IN, H, W)
  K_init = np.random.randn(K, K, C_IN, C_OUT)
  b_init = np.random.randn(C_OUT)
  y = layers.Conv2D(C_OUT, K, strides=S, padding=padding, data_format='channels_first',
    dilation_rate=1, groups=1, activation=None, use_bias=True, input_shape=(C_IN, H, W))
  y_keras = y(x).numpy()
  y.set_weights([K_init, b_init])
  y_keras = y(x).numpy()
  y_out = Conv2DLayer(kernel_size=K, input_channels=C_IN, output_channels=C_OUT, 
                 padding=padding, stride=S, K_init=K_init, b_init=b_init).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Conv2D 2')
  return

In [306]:
test_Conv2DLayer()

Test Conv2D 1 Passed!
Test Conv2D 2 Passed!




---



* (2 балла) Реализация **транспонированного сверточного** слоя

In [307]:
class Conv2DTrLayer(Layer):
    def __init__(self, kernel_size=3, input_channels=2, output_channels=3, 
                 padding=0, stride=1, K_init=None, b_init=None):      
      # padding: число (сколько отрезать от модифицированной входной карты)
      # Работаем с квадратными ядрами, поэтому kernel_size - одно число
      # stride - одно число (коэффициент расширения)
      # Фильтр размерности [kernel_size, kernel_size, input_channels, output_channels]
      self.name = 'Conv2DTr'
      self.kernel_size = kernel_size
      self.input_channels = input_channels
      self.output_channels = output_channels
      self.kernel = K_init
      self.bias = b_init
      self.padding = padding
      self.stride = stride
    def forward(self, input_data):
      # На входе - четырехмерный тензор вида [batch, input_channels, height, width]
      # Вначале нужно проверить на согласование размерностей входных данных и ядра!
      assert input_data.shape[1] == self.input_channels, "alert!!! different shapes"
      # Нужно заполнить Numpy-тензор out 
      out = np.zeros((input_data.shape[0],self.output_channels,(input_data.shape[2]-1)*self.stride-2*self.padding+self.kernel_size,(input_data.shape[3]-1)*self.stride-2*self.padding+self.kernel_size))

      input_data_converted = np.zeros((input_data.shape[0],self.input_channels,(input_data.shape[2]-1)*self.stride+2*(self.kernel_size-self.padding)-1,(input_data.shape[3]-1)*self.stride+2*(self.kernel_size-self.padding)-1))

      ##расширяем
      padingg = self.kernel_size - self.padding - 1
      for batch in range(input_data.shape[0]):
        for in_k in range(self.input_channels):
          input_data_converted[batch,in_k,padingg:-padingg:self.stride,padingg:-padingg:self.stride] = input_data[batch,in_k]

    ##сворачиваем

        
      input_data = input_data_converted
    
      for out_k in range(self.output_channels):
        for batch in range(input_data.shape[0]):
          for i in range(0,input_data.shape[2] - self.kernel_size+1):
            for j in range(0,input_data.shape[3]- self.kernel_size+1):
              for in_k in range(self.input_channels):
                out[batch,out_k,i ,j] += np.sum(self.kernel[:,:,in_k,out_k] * input_data[batch,in_k,i : (i + self.kernel_size),j:(j + self.kernel_size)])

          out[batch,out_k] += self.bias[out_k]

      return out

In [308]:
def adjust_kernel(K):
  K_new = K.copy()[::-1, ::-1, :, :]
  K_new = np.transpose(K_new, (0, 1, 3, 2))
  return K_new

def test_Conv2DTrLayer():
  B = 1
  C_IN = 1
  C_OUT = 1
  H = 3
  W = 3
  K = 2
  S = 2
  padding = 0
  x = np.random.randn(B, C_IN, H, W)
  K_init = np.random.randn(K, K, C_IN, C_OUT)
  b_init = np.random.randn(C_OUT)
  y = layers.Conv2DTranspose(C_OUT, K, strides=S, padding="valid", output_padding=None, 
                    data_format='channels_first', dilation_rate=1, groups=1, 
                    activation=None, use_bias=True)
  y_keras = y(x).numpy()
  y.set_weights([adjust_kernel(K_init), b_init])
  y_keras = y(x).numpy()
  y_out = Conv2DTrLayer(kernel_size=K, input_channels=C_IN, output_channels=C_OUT, 
                 padding=padding, stride=S, K_init=K_init, b_init=b_init).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Conv2DTr 1')
  B = 4
  C_IN = 2
  C_OUT = 3
  H = 3
  W = 3
  K = 3
  S = 2
  padding = 0
  x = np.random.randn(B, C_IN, H, W)
  K_init = np.random.randn(K, K, C_IN, C_OUT)
  b_init = np.random.randn(C_OUT)
  y = layers.Conv2DTranspose(C_OUT, K, strides=S, padding="valid", output_padding=None, 
                    data_format='channels_first', dilation_rate=1, groups=1, 
                    activation=None, use_bias=True)
  y_keras = y(x).numpy()
  y.set_weights([adjust_kernel(K_init), b_init])
  y_keras = y(x).numpy()
  y_out = Conv2DTrLayer(kernel_size=K, input_channels=C_IN, output_channels=C_OUT, 
                 padding=padding, stride=S, K_init=K_init, b_init=b_init).forward(x)
  compare_tensors(y_keras, y_out, tol=0.001, test_name='Test Conv2DTr 2')
  return

In [309]:
test_Conv2DTrLayer()

Test Conv2DTr 1 Passed!
Test Conv2DTr 2 Passed!
